In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import glob
import re

In [2]:
ecad_custom = requests.get('https://www.ecad.eu/dailydata/customquery.php?optionSelected=element&processtext1=Your+query+is+being+processed.+Please+wait...&blendingselect=yes&countryselect=GERMANY%7Cde&stationselect=All+stations%7C**&elementselect=Mean+temperature%7Ctg&processtext2=Your+query+is+being+processed.+Please+wait...')

In [3]:
# requests.text will return the html file of the website as a string

In [4]:
# print(ecad_custom.text) # hmtl string

In [5]:
soup = BeautifulSoup(ecad_custom.text, 'lxml')
# print(soup)

In [6]:
# get all berlin station names
s = soup.find('select', attrs={"name":"stationselect"})
berlin_stations = []
for option in s.find_all('option'):
    station_name = option.get('value')
    if 'BERLIN-' in station_name:
        berlin_stations.append(station_name)

print(berlin_stations)
    

['BERLIN-ADLERSHOF[de]|11736', 'BERLIN-BUCH[de]|4529', 'BERLIN-DAHLEM (LFAG)[de]|4533', 'BERLIN-DAHLEM[de]|41', 'BERLIN-FRIEDRICHSHAGEN[de]|11737', 'BERLIN-FROHNAU[de]|11738', 'BERLIN-INVALIDENSTRASSE[de]|11739', 'BERLIN-KANISWALL[de]|4546', 'BERLIN-LICHTENRADE[de]|4556', 'BERLIN-LICHTERFELDE (SUD)[de]|4559', 'BERLIN-MARZAHN[de]|4561', 'BERLIN-MITTE[de]|4563', 'BERLIN-OSTKREUZ[de]|11740', 'BERLIN-RUDOW[de]|4566', 'BERLIN-SCHONEFELD[de]|4570', 'BERLIN-SPANDAU[de]|4575', 'BERLIN-TEGELER FLIESSTAL[de]|4581', 'BERLIN-TEGEL[de]|4005', 'BERLIN-TEMPELHOF[de]|2759', 'BERLIN-TREPTOW[de]|4586', 'BERLIN-ZEHLENDORF[de]|4588']


In [7]:
# remove the first few lines from the txt files

In [8]:
for file in os.listdir('data/ECA_blended_custom/'):
    #print(file)
    if file.startswith('TG_STAID'):
        current_file = file
        #print(current_file)
        
        with open('data/ECA_blended_custom/' + current_file, "r") as f:
            content = f.readlines()
            content = content[18:]
            #print(content)
            #print(type(content)) # is a list
            string ='\n'.join(content)
            new_f = open(f"data/berlin_temp_clean/cleaned_{current_file}", "w") 
            new_f.write(string)

In [9]:
# import all temperature files and concatenate to one big dataframe
path = 'data/berlin_temp_clean/' 
all_files = glob.glob(path + "clean*.txt")

temp_file_list = []

for filename in all_files:
    
    # find station number in filename
    station = re.findall('(0+)(\d*)', filename)[0][1]
    station_name = ""
    
    # match station number with station name from "BeautifulSoup" list 
    for berlin_station in berlin_stations:
        berlin_station_name = re.match('\w{6}\-\w{2,}[^\[]', berlin_station)
        if station in berlin_station:
            station_name = berlin_station_name.group(0)

    single_df = pd.read_csv(filename, index_col=None, header=0).assign(Station=station, Station_name=station_name)
    
    temp_file_list.append(single_df)

df = pd.concat(temp_file_list, axis=0, ignore_index=True)

In [10]:
df.head()

,SOUID,DATE,TG,Q_TG,Station,Station_name
0,127488,18760101,22,0,4005,BERLIN-TEGEL
1,127488,18760102,25,0,4005,BERLIN-TEGEL
2,127488,18760103,3,0,4005,BERLIN-TEGEL
3,127488,18760104,-58,0,4005,BERLIN-TEGEL
4,127488,18760105,-98,0,4005,BERLIN-TEGEL


In [11]:
df['Station'].value_counts

<bound method IndexOpsMixin.value_counts of 0         4005
1         4005
2         4005
3         4005
4         4005
          ... 
712679    4586
712680    4586
712681    4586
712682    4586
712683    4586
Name: Station, Length: 712684, dtype: object>

In [12]:
df.shape

(712684, 6)

In [13]:
df.to_csv('data/berlin_temp_clean/berlin_all_districts_temp.csv', index=True)